# Interactive Face Detection Sample

This sample showcases Object Detection task applied for face recognition using sequence of neural networks.
Async API usage can improve overall frame-rate of the application, because rather than wait for inference to complete,
the application can continue operating on the host while accelerator is busy.
This sample maintains three parallel infer requests for the Age/Gender Recognition, Head Pose Estimation, and Emotions Recognition that run simultaneously.

Other sample objectives are:

*	Video as input support via OpenCV
*	Visualization of the resulting face bounding boxes from Face Detection network
*	Visualization of age/gender, head pose and emotion information for each detected face

OpenCV\* is used to draw resulting bounding boxes, labels, and other information. You can copy and paste this code without pulling Inference Engine sample helpers into your application

## How it Works

*	The application reads command line parameters and loads up to four networks depending on `-m...` options family to the Inference Engine.
*	The application gets a frame from the OpenCV's VideoCapture.
*	The application performs inference on the frame detection network.
*	The application performs three simultaneous inferences, using the Age/Gender, Head Pose and Emotions detection networks if they are specified in command line.
*	The application displays the results.

The new Async API operates with a new notion of the Infer Request that encapsulates the inputs/outputs and separates scheduling and waiting for result. For more information about Async API and the difference between Sync and Async modes performance, refer to **How it Works** and **Async API** sections in [Object Detection SSD, Async API Performance Showcase Sample](@ref InferenceEngineObjectDetectionSSDDemoAsyncApplication).


## Compiling

There are two source files: 
* [detectors.cpp](detectors.cpp) -- contains the various helper functions
* [main.cpp](main.cpp) -- contains the main loop

We have provided a Makefile for compiling the application. Run the following cell to run the make command.

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))
from demoTools.demoutils import *

In [ ]:
!make

This produces an executable [face_detector](). This executable takes in a number of different command line arguments.

Run the following cell to see the list: 

In [ ]:
%%bash
export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/data/reference-sample-data/extension/ 
./face_detector -h

The version of the cpp file here is a slightly modified version of the faca_detector code built-in to the OpnVINO distribution.
In this version, the result is written into a output mp4 file specified by the `-o` flag. 

### Running the inference

Now we are ready to run the inference workload. In this step we will be submitting the workload as a job to the job queue.

Currently, you are on what is called a "devnode". On this system, you are alloated just one core on a large Xeon CPU. The purpose of this node is to develop code and run minimal jupyter notebooks, but it is not meant for compute jobs like deep learning inference. So we need to request additional resources from the cluster to run the inference, and this is done through the job queue.

To put an item on the job queue, we must first create a bash script that run the workload we want. Run the following cell to create bash script [face_detection_demo.sh](face_detection_demo.sh) which will be our job script. 

In [ ]:
!/opt/intel/computer_vision_sdk/deployment_tools/model_downloader/downloader.py --print_all

In [ ]:
!/opt/intel/computer_vision_sdk/deployment_tools/model_downloader/downloader.py --name mobilenet-ssd -o raw_models

In [ ]:
!/opt/intel/computer_vision_sdk/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP32 \
-o models/mobilenet-ssd/FP32 \
--scale 256 \
--mean_values [127,127,127] 

In [ ]:
!/opt/intel/computer_vision_sdk/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP16 \
-o models/mobilenet-ssd/FP16 \
--scale 256 \
--mean_values [127,127,127] 

In [ ]:
%%writefile face_detection_demo.sh
#PBS
if [ "$2" == "MYRIAD" ]; then
  FP="FP16"
else
  FP="FP32"
fi

if [ "$2" = "HETERO:FPGA,CPU" ]; then
    source /opt/intel/computer_vision_sdk/bin/setup_hddl.sh
    aocl program acl0 /opt/intel/computer_vision_sdk_2018.4.420/bitstreams/a10_vision_design_bitstreams/4-0_PL1_FP11_MobileNet_ResNet_VGG_Clamp.aocx
fi

cd $PBS_O_WORKDIR
export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/data/reference-sample-data/extension/ 
MODEL_ROOT=/opt/intel/computer_vision_sdk/deployment_tools/intel_models/
./face_detector -i $1 -no_wait \
-m ${MODEL_ROOT}/face-detection-adas-0001/$FP/face-detection-adas-0001.xml \
-m_hp ${MODEL_ROOT}/head-pose-estimation-adas-0001/$FP/head-pose-estimation-adas-0001.xml \
-m_ag ${MODEL_ROOT}/age-gender-recognition-retail-0013/$FP/age-gender-recognition-retail-0013.xml \
-m_em ${MODEL_ROOT}/emotions-recognition-retail-0003/$FP/emotions-recognition-retail-0003.xml \
-d $2 \
-o $3

To put this script on the job queue, we use the command `qsub`.
There are two important arguments we use with this command.

First, the `-l` flag.
This flag is used to specify what type of resources to request from the cluster.
For example this can be used to request a Intel Xeon system, or it can be used to request a system with an FPGA.
The syntax is `-l nodes=1:<tag>` where `<tag>` is the descriptor tag for the resource you want.
For example, `-l nodes=1:iei-tank-xeon` will request an Intel Xeon system.
To see the list of available tags, and the number of avilable systems, run the following cell.

In [ ]:
!pbsnodes | grep properties | sort | uniq -c

Then there is the `-F` flag, which is used to pass in arguments to the job script.
The [face_detection_demo.sh](face_detection_demo.sh) takes in 2 arguments:
1) the path to the video to run inference on
2) targeted device (CPU,GPU,MYRIAD)
The job scheduler will use the contents of `-F` flag as the argument to the job script.

The following line will request an Intel Xeon system, and passes in "faces-recognition-walking.mp4 CPU" to the job script. Run the cell to submit this job. 

#### submitting to a node with Intel Xeon CPU

In [ ]:
print("Submitting job to Intel Xeon CPU...")
#Submit job to the queue
job_id_xeon = !qsub face_detection_demo.sh -l nodes=1:iei-tank-xeon -F "faces-recognition-walking-and-pause.mp4 CPU results/xeon"
print(job_id_xeon[0])

#Progress indicators
if job_id_xeon:
    progressIndicator('results/xeon', 'i_progress_'+job_id_xeon[0]+'.txt', "Inference", 0, 100)

Here you should see a output like "{job_id}.c003", where {job_id} is a number.
This is your job ID, and this value can be used to check on the progress of the job down the line.
Furthermore, the above job script has been written so that it uses its job_id as the name of the output video.   

One bigadvantage of the Job queue system is that you may submit multiple jobs at once. 
These jobs will be run as soon as resources are available, and may all run at once if the cluster is not busy.
Here are few other "preset" jobs that for your convenience that you can run. 

#### submitting to a node with Intel Core CPU

In [ ]:
print("Submitting job to Intel Core CPU...")
#Submit job to the queue
job_id_core = !qsub face_detection_demo.sh -l nodes=1:iei-tank-core -F "faces-recognition-walking.mp4 CPU results/core"
print(job_id_core[0])

#Progress indicators
if job_id_core:
    progressIndicator('results/core', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)

#### submitting to a node with Intel Core CPU and using the onboard Intel GPU

In [ ]:
print("Submitting job to Intel Core CPU with Intel GPU...")
#Submit job to the queue
job_id_gpu = !qsub face_detection_demo.sh -l nodes=1:iei-tank-core -F "head-pose-female-male.mp4 GPU results/gpu"
print(job_id_gpu[0])

#Progress indicators
if job_id_gpu:
    progressIndicator('results/gpu', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)

#### submitting to a node with Intel Movidius NCS (Neural Computing Stick)

In [ ]:
print("Submitting job to node with Intel Movidius NCS...")
#Submit job to the queue
job_id_myriad = !qsub face_detection_demo.sh -l nodes=1:iei-tank-movidius -F "faces-recognition-walking.mp4 MYRIAD results/myriad"
print(job_id_myriad[0])

#Progress indicators
if job_id_myriad:
    progressIndicator('results/myriad', 'i_progress_'+job_id_myriad[0]+'.txt', "Inference", 0, 100)

#### submitting to a node with Intel FPGA HDDL-F (High Density Deep Learning)

In [ ]:
print("Submitting job to node with Intel FPGA HDDL-F...")
#Submit job to the queue
job_id_fpga = !qsub face_detection_demo.sh -l nodes=1:iei-tank-fpga -F "head-pose-female-male.mp4 HETERO:FPGA,CPU results/fpga"
print(job_id_fpga[0])

#Progress indicators
if job_id_fpga:
    progressIndicator('results/fpga', 'i_progress_'+job_id_fpga[0]+'.txt', "Inference", 0, 100)

The `qstat` command is used to track the progress of the jobs. 
We have provided a utility funtion "liveQstat()" that provide a live updating GUI for you.
Run the following cell to check on the progress of your earlier jobs.

In [ ]:
liveQstat()

Once the jobs are done. The following cell can be used to display the output. 
The videoHTML is a utility fnction provided in [demoutils.py](demoutils.py).
This takes one argument, which is the path to the video (see above for convention for the path name).

For your convenience we have stored the jobid from the above `qsub` commands, and the following cells 

In [ ]:
videoHTML('IEI Tank Xeon (Intel Xeon CPU)',['results/xeon/output_'+job_id_xeon[0]+'.mp4'], 'results/xeon/stats.txt')

In [ ]:
videoHTML('IEI Tank (Intel Core CPU)', ['results/core/output_'+job_id_core[0]+'.mp4'], 'results/core/stats.txt')

In [ ]:
videoHTML('IEI Intel GPU (Intel Core + Onboard GPU)', ['results/gpu/output_'+job_id_gpu[0]+'.mp4'], 'results/gpu/stats.txt')

In [ ]:
videoHTML('IEI Tank + Intel FPGA HDDL-F',['results/fpga/output_'+job_id_fpga[0]+'.mp4'], 'results/fpga/stats.txt')

In [ ]:
videoHTML('IEI Tank + (Intel CPU + Movidius)',['results/myriad/output_'+job_id_myriad[0]+'.mp4'], 'results/myriad/stats.txt')

In [ ]:
summaryPlot({'results/core/stats.txt':'Core', 'results/xeon/stats.txt':'Xeon', 'results/gpu/stats.txt':'GPU', 'results/fpga/stats.txt':'FPGA', 'results/myriad/stats.txt':'Myriad'}, 'Architecture', 'Time, seconds', 'Inference engine processing time' )